<a href="https://colab.research.google.com/github/dxda6216/ttron2excel/blob/main/ttron_data_file_to_excel_file_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Converting Ttron TRACES file (TRACES.nnn) to Excel file {single-column: true}
#
# This is a simple script for coverting a Taylortron data file (the Johnson
# Lab, 30 channels, "TRACES.nnn" file) to an Excel file, detrending, smoothing,
# and plotting the data.
# by dxda6216 (Email: dxda6216 AT gmail DOT com)
#
import pandas as pd
import csv
import math
import numpy as np
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MultipleLocator
import io
from google.colab import files
from scipy import signal # Added for sinc filter implementation

#@markdown **This script works only with a specific format of data files (*TRACES.nnn* files) generated by the Taylortron in the Johnson Lab.**

Experiment_number = 'CYxxx' #@param {type:"string"}
Experiment_title = '' #@param {type:"string"}
Date_experiment_started = '2026-01-01' #@param {type:"date"}

Detrending_Method = "Sinc Filter" #@param ["Moving Average", "Sinc Filter"]

# For detrending by moving average, ste an window size
Window_size_for_trend_line_moving_average = 24 #@param {type:"number"}

# Sinc Filter Parameters (only used if 'Sinc Filter' is selected)
Sinc_Filter_Cutoff_Period_Hours = 24 #@param {type:"number"}
Sinc_Filter_Order = 101 #@param {type:"number"}

Data_plotting = "Plotting the channel 00 data last" #@param ["Plotting the channel 00 data first", "Plotting the channel 00 data last"]

# Plotting the data from the channel 0 or from the channel 1
if Data_plotting == "Plotting the channel 00 data first":
	chlist = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
else:
	chlist = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,0]

#@markdown 1. Input the experiment number (avoid spaces and special characters).
#@markdown 2. Input the experiment title (this field can be blank).
#@markdown 3. Input the date on which the experiment started.
#@markdown 4. Select 'Moving Average' or 'Sinc Filter' for detrending.
#@markdown 5. If 'Moving Average' is chosen, adjust 'Window_size_for_trend_line_moving_average'.
#@markdown 6. If 'Sinc Filter' is chosen, adjust 'Sinc_Filter_Cutoff_Period_Hours' and 'Sinc_Filter_Order'.
#@markdown 7. **Runtime** -> **Restart and run all** (or press **Ctrl+M** and then press **Ctrl+F9**)
#@markdown 8. Wait until `Choose Files` or `Browse...` button appears below.
#@markdown 9. Click `Choose Files` or `Browse...` button and select *TRACES.nnn* file in your computer.
#@markdown 10. Wait a while. two Excel files, one ZIP file, and one PDF file will be saved in "Downloads" folder in your computer.
#@markdown - The first Excel file will have multiple spreadsheets, conatining all the raw data, smoothed data, and detranded data.
#@markdown - The second Excel file will have a single spreadsheet, conatining only the raw time series data. The interval time will be indicated as a sheet name of the Excel spreadsheet. This file can be opened with data analysis programs such as [pyBOAT](https://github.com/tensionhead/pyBOAT).
#@markdown - The ZIP file will contain separate data files (.dat files) for each of the channels. The .dat files can be opened with the [LumiCycle](https://actimetrics.com/products/lumicycle/) Analysis program.

### Deleting old data files
!rm -r -f *.xlsx *.pdf *.dat *.zip TRACES.* Traces.* traces.*

### Uploading TRACES.xxx file
uploaded = files.upload()
ttronfilename = next(iter(uploaded))

### Reading the uploaded TRACES file to a dataframe
print('\nReading the data...')

colnames = ["Hours","00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29"]

df = pd.read_csv(ttronfilename, header=None, sep ='\t', skiprows=3, skipfooter=1, index_col=False, names=colnames, engine='python')
df2= df.iloc[:,0:31]

print(df)
print(df2)

number_of_rows = len(df.index)
last_row_index = number_of_rows - 1
print('Number of rows: ' + str(number_of_rows))
print('First time points: ' + str(df.loc[0]['Hours']) + ' h')
print('Last time points: ' + str(df.loc[last_row_index]['Hours']) + ' h')
total_time = df.loc[last_row_index]['Hours'] - df.loc[0]['Hours']
total_time_in_days = total_time * (1/24)
print('Total time duration: ' + str(total_time) + ' h = ' + str(total_time_in_days) + ' days')
time_interval = total_time / last_row_index
excel2_sheet_name = 'INTVL = '+str('{:.15f}'.format(time_interval))+' h'
print('Average time interval: ' + str(time_interval) + ' h')

####### Calculating moving averages and trend line #######
print('\nCalculating trend line and detrended data using ' + Detrending_Method + '...')

df_5PMA = df.rolling(window=5, center=True).mean()
df_9PMA = df.rolling(window=9, center=True).mean()

if Detrending_Method == "Moving Average":
    ###### Moving average time window for trend line
    tws = math.ceil(Window_size_for_trend_line_moving_average/time_interval)
    if tws%2 == 0:
    	tws = tws + 1

    twss = int(tws)
    twst = time_interval * (twss - 1)
    print('Window size for trend line (Moving Average): ' + str(twss) + ' points (' + str('{:.6f}'.format(twst)) + ' h)')
    df_TL_MA = df.rolling(window=twss, center=True).mean()
    trendline_sheet_name = 'Trend line ('+str(twss)+'PMA)'

elif Detrending_Method == "Sinc Filter":
    # Sinc Filter Parameters Calculation
    sampling_rate = 1 / time_interval # samples per hour
    cutoff_frequency = 1 / Sinc_Filter_Cutoff_Period_Hours # cycles per hour
    nyquist_frequency = 0.5 * sampling_rate
    norm_cutoff = cutoff_frequency / nyquist_frequency
    filter_order = Sinc_Filter_Order

    # Ensure filter order is not too large for filtfilt
    max_filter_order_for_data = int(number_of_rows / 3) - 1
    if filter_order >= max_filter_order_for_data:
        print(f"Warning: Sinc Filter Order ({filter_order}) is too high for data length ({number_of_rows} points).")
        filter_order = max_filter_order_for_data
        if filter_order % 2 == 0: # Ensure filter order is odd
            filter_order -= 1 # Reduce by 1 if even
        if filter_order <= 0: # Ensure it's at least 1 if data is very short
            filter_order = 1
        print(f"Adjusting Sinc Filter Order to {filter_order} for stability with filtfilt.")

    if filter_order % 2 == 0: # Ensure filter order is odd for filtfilt stability
        filter_order += 1

    print(f"Sampling Rate: {sampling_rate:.2f} samples/hour")
    print(f"Cutoff Period for Sinc Filter: {Sinc_Filter_Cutoff_Period_Hours} hours")
    print(f"Cutoff Frequency: {cutoff_frequency:.4f} cycles/hour")
    print(f"Normalized Cutoff Frequency: {norm_cutoff:.4f}")
    print(f"Sinc Filter Order: {filter_order}")

    # Design the FIR filter (sinc-like filter)
    sinc_filter_coeffs = signal.firwin(filter_order, norm_cutoff, pass_zero='lowpass')

    # Create a new DataFrame for sinc-derived trend
    df_TL_MA = df.copy() # Use df_TL_MA for the trend output for consistency

    # Apply the filter to each channel to get the trend
    for k in range(0, 30, 1):
        channelnumber = str(k).zfill(2)
        if df[channelnumber].isnull().all(): # Skip if channel data is all NaN
            df_TL_MA[channelnumber] = np.nan
            continue

        # Handle NaNs: interpolate or fill before filtering.
        # Using linear interpolation for better results than mean fill.
        data_to_filter = df[channelnumber].interpolate(method='linear', limit_direction='both', axis=0)

        # Apply the filter using filtfilt for zero-phase distortion
        # Use try-except to catch potential issues with short data or filter order
        try:
            df_TL_MA[channelnumber] = signal.filtfilt(sinc_filter_coeffs, [1.0], data_to_filter)
        except ValueError as e:
            print(f"Warning: Could not apply sinc filter to channel {channelnumber}. Error: {e}")
            print("Consider reducing filter_order or increasing data length.")
            df_TL_MA[channelnumber] = np.nan # Set to NaN if filtering fails

    trendline_sheet_name = f'Trend line (Sinc Filter C={Sinc_Filter_Cutoff_Period_Hours}h O={filter_order})'

# Detrend the data by subtracting the trend line
dtdf = df - df_TL_MA
dtdf['Hours'] = df_TL_MA['Hours'] # Keep the 'Hours' column intact
dtdf_5PMA = dtdf.rolling(window=5, center=True).mean()
dtdf_9PMA = dtdf.rolling(window=9, center=True).mean()

### Generating an Excel file containing all the data
print('\nGenerating an Excel file...')
now = datetime.now(timezone.utc)
processed_dnt_str = now.strftime("%Y-%m-%d %H:%M:%S")
note_df = pd.DataFrame.from_dict(
	{
		'A': ['Experiment Number', 'Experiment Title', 'Experiment Start Date', 'TRACES File', '', 'Number of Time Points', 'Total Time Duration (Hours)', 'Average Time Interval (Hours)', 'Detrending Method', 'Trend Line Parameter', '', 'Data Processed Date and Time (UTC)'] ,
		'B': ['', '', '', '', '', '', '', '', '', '', '', ''],
		'C': ['', '', '', '', '', '', '', '', '', '', '', ''],
		'D': ['', '', '', '', '', '', '', '', '', '', '', ''],
		'E': [Experiment_number, Experiment_title, Date_experiment_started, ttronfilename, '', number_of_rows, total_time, time_interval, Detrending_Method, (f'{Window_size_for_trend_line_moving_average}h window' if Detrending_Method == 'Moving Average' else f'{Sinc_Filter_Cutoff_Period_Hours}h cutoff, {Sinc_Filter_Order} order'), '', processed_dnt_str]
	}
)

outputexcelfilename = Experiment_number+"_data.xlsx"
with pd.ExcelWriter(outputexcelfilename) as writer:
	note_df.to_excel(writer, sheet_name='Note', index=None, header=False)
	df.to_excel(writer, sheet_name='Raw Data')
	df_5PMA.to_excel(writer, sheet_name='5-point moving average (5PMA)')
	df_9PMA.to_excel(writer, sheet_name='9-point moving average (9PMA)')
	df_TL_MA.to_excel(writer, sheet_name=trendline_sheet_name)
	dtdf.to_excel(writer, sheet_name='Detrended Data')
	dtdf_5PMA.to_excel(writer, sheet_name='Detrended Data 5PMA')
	dtdf_9PMA.to_excel(writer, sheet_name='Detrended Data 9PMA')
	for k in range(0, 30, 1):
		channelnumber = str(k).zfill(2)
		chnum = 'Channel '+channelnumber
		dfx = pd.DataFrame()
		dfx['Hours'] = df['Hours']
		dfx['Raw_data'] = df[channelnumber]
		dfx['5PMA'] = df_5PMA[channelnumber]
		dfx['9PMA'] = df_9PMA[channelnumber]
		dfx['trend_line'] = df_TL_MA[channelnumber]
		dfx['detrended_data'] = dtdf[channelnumber]
		dfx['detrended_data_5PMA'] = dtdf_5PMA[channelnumber]
		dfx['detrended_data_9MPA'] = dtdf_9PMA[channelnumber]
		dfx.to_excel(writer, sheet_name=chnum)

print('\nExcel file: '+outputexcelfilename+'  has been generated.')

### Generating an Excel file conatining only the raw data
outputexcelfilename2 = Experiment_number+"_data_2.xlsx"
with pd.ExcelWriter(outputexcelfilename2) as writer:
	df2.to_excel(writer, sheet_name=excel2_sheet_name, index=None, header=True)

print('\nExcel file: '+outputexcelfilename2+'  has been generated.')

### Generating .dat files
print('\nGenerating a data file for each channel (.dat files)...')
df['Days'] = df['Hours'] / 24.000
for k in range(0, 30, 1):
	channelnumber = str(k).zfill(2)
	datfilename = channelnumber + '.dat'
	df.to_csv(datfilename, header=False, index=False, sep ='\t', columns=['Days',channelnumber])

### Packing all the .dat files into a zip file
print('\nPacking .dat files into a zip file...')
zip_output_filename = Experiment_number + '_data.zip'
!zip -r {zip_output_filename} ./*.dat

print('\nPlotting...')
plot_output_pdf = Experiment_number + "_data_plots.pdf"
x = df['Hours']
x_scale_min = int(math.floor(min(x)*(1/24)))*24
x_scale_max = int(math.ceil(max(x)*(1/12)))*12+12
xtickslist = list(range(x_scale_min, x_scale_max, 12))

pp = PdfPages(plot_output_pdf)
plt.rcParams.update({'figure.max_open_warning': 0})

fig = plt.figure(figsize=(11, 8.5))
fig.subplots_adjust(hspace=0.15)

plt.rc('font', size=5)
plt.rc('axes', titlesize=4)
plt.rc('axes', labelsize=4)
plt.rc('xtick', labelsize=4)
plt.rc('ytick', labelsize=4)
plt.rc('legend', fontsize=3)
plt.rc('figure', titlesize=5)

subplotnumber = 1
for k in chlist:
		channelnumber = str(k).zfill(2)
		plt.subplot(10, 3, subplotnumber)
		x = df['Hours']
		y = df[channelnumber]
		x_cma = df_TL_MA['Hours']
		y_cma = df_TL_MA[channelnumber]

		print('Plotting Channel '+channelnumber)
		chlabel = 'Ch # '+channelnumber
		plt.scatter(x,y,s=0.1,c='blue', label=chlabel)
		plt.plot(x_cma,y_cma,'-r', linewidth=0.5, label='trend line ('+Detrending_Method+')')
		plt.xlim(x_scale_min-6, x_scale_max)
		y_scale_max = int(max(y)*1.100)
		plt.ylim(0, y_scale_max)
		plt.xticks(xtickslist)
		plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
		plt.legend(loc='upper right', fontsize=4)
		plt.suptitle(Experiment_number, fontsize=12)
		subplotnumber += 1

fig.text(0.50, 0.08, 'Time (hours)', horizontalalignment='center', fontsize = 10)
fig.text(0.08, 0.50, 'Bioluminescence', horizontalalignment='center', verticalalignment='center', rotation='vertical', fontsize = 10)
pp.savefig(fig)

fig = plt.figure(figsize=(11, 8.5))
fig.subplots_adjust(hspace=0.15)

subplotnumber = 1
for k in chlist:
		channelnumber = str(k).zfill(2)
		plt.subplot(10, 3, subplotnumber)
		y = df[channelnumber]
		x_dt = dtdf['Hours']
		y_dt = dtdf[channelnumber]
		x_dt_cma = dtdf_5PMA['Hours']
		y_dt_cma = dtdf_5PMA[channelnumber]
		chlabel = 'Ch # '+channelnumber+' - detrend'
		print('Plotting Channel '+channelnumber+' (detrended)')
		plt.scatter(x_dt,y_dt,s=0.1,c='blue', label=chlabel)
		plt.plot(x_dt_cma,y_dt_cma,'-r', linewidth=0.5)
		plt.xlim(x_scale_min-6, x_scale_max)
		plt.xticks(xtickslist)
		plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
		plt.legend(loc='upper right', fontsize=4)
		plt.suptitle(Experiment_number+' - detrended data ('+Detrending_Method+')', fontsize=12)
		subplotnumber += 1

fig.text(0.50, 0.08, 'Time (hours)', horizontalalignment='center', fontsize = 10)
fig.text(0.08, 0.50, 'Detrended Bioluminescence', horizontalalignment='center', verticalalignment='center', rotation='vertical', fontsize = 10)
pp.savefig(fig)

print('\nPlotting...')
x = df['Hours']
x_scale_min = int(math.floor(min(x)*(1/24)))*24
x_scale_max = int(math.ceil(max(x)*(1/12)))*12+12
xtickslist = list(range(x_scale_min, x_scale_max, 12))

plt.rc('font', size=10)
plt.rc('axes', titlesize=10)
plt.rc('axes', labelsize=10)
plt.rc('xtick', labelsize=9)
plt.rc('ytick', labelsize=9)
plt.rc('legend', fontsize=12)
plt.rc('figure', titlesize=14)

for k in chlist:
	fig = plt.figure(figsize=(11, 8.5))
	channelnumber = str(k).zfill(2)
	x = df['Hours']
	y = df[channelnumber]
	x_5cma = df_5PMA['Hours']
	y_5cma = df_5PMA[channelnumber]
	print('Plotting Channel '+channelnumber)
	plt.scatter(x,y,s=5.0,c='violet', label='Bioluminescence')
	plt.plot(x_5cma,y_5cma,'-r', linewidth=1.0, label='Smoothed line (5-point moving average, centered)')
	plt.xlim(x_scale_min, x_scale_max)
	y_scale_max = int(max(y)*1.100)
	plt.ylim(0, y_scale_max)
	plt.xticks(np.arange(x_scale_min, x_scale_max, 12.0))
	plt.xlabel('Hours', fontsize=12)
	plt.ylabel('Bioluminescence', fontsize=12)
	plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
	plt.legend(loc='upper right', fontsize=6)
	plt.suptitle(Experiment_number+'   Ch # '+channelnumber, fontsize=14)
	pp.savefig(fig)

plt.rc('font', size=10)
plt.rc('axes', titlesize=10)
plt.rc('axes', labelsize=10)
plt.rc('xtick', labelsize=9)
plt.rc('ytick', labelsize=9)
plt.rc('legend', fontsize=6)
plt.rc('figure', titlesize=14)

print('\nPlotting...')

for k in chlist:
	fig = plt.figure(figsize=(11, 8.5))
	channelnumber = str(k).zfill(2)
	x = df['Hours']
	y = df[channelnumber]
	x_cma = df_TL_MA['Hours']
	y_cma = df_TL_MA[channelnumber]
	x_dt = dtdf['Hours']
	y_dt = dtdf[channelnumber]
	x_dt_cma = dtdf_5PMA['Hours']
	y_dt_cma = dtdf_5PMA[channelnumber]

	plt.subplot(2, 1, 1)
	print('Plotting Channel '+channelnumber)
	plt.scatter(x,y,s=5.0,c='violet', label='Bioluminescence')

	if Detrending_Method == "Moving Average":
	    plt.plot(x_cma,y_cma,'-r', linewidth=1.0, label='Trend line (' + str(twss) +'-point moving average, centered)')
	else: # Sinc Filter
	    plt.plot(x_cma,y_cma,'-r', linewidth=1.0, label=f'Trend line (Sinc Filter C={Sinc_Filter_Cutoff_Period_Hours}h O={filter_order})')

	plt.xlim(x_scale_min, x_scale_max)
	y_scale_max = int(max(y)*1.100)
	plt.ylim(0, y_scale_max)
	plt.xticks(np.arange(x_scale_min, x_scale_max, 12.0))
	plt.xlabel('Hours', fontsize=12)
	plt.ylabel('Bioluminescence', fontsize=12)
	plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
	plt.legend(loc='upper right', fontsize=6)

	plt.subplot(2, 1, 2)
	print('Plotting Channel '+channelnumber+' (detrended)')
	plt.scatter(x_dt,y_dt,s=5.0,c='violet', label='Detrended bioluminescence')
	plt.plot(x_dt_cma,y_dt_cma,'-b', linewidth=1.0, label='Smoothed line (5-point moving average, centered)')
	plt.xlim(x_scale_min, x_scale_max)
	plt.xticks(np.arange(x_scale_min, x_scale_max, 12.0))
	plt.xlabel('Hours', fontsize=12)
	plt.ylabel('Detrended bioluminescence', fontsize=12)
	plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
	plt.legend(loc='upper right', fontsize=6)
	plt.suptitle(Experiment_number+'   Ch # '+channelnumber, fontsize=14)
	pp.savefig(fig)

pp.close()

files.download(outputexcelfilename)
files.download(outputexcelfilename2)
files.download(plot_output_pdf)
files.download(zip_output_filename)
